Created by [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [Cumberland Farms](https://www.cumberlandfarms.com/stores).
<hr>

# Cumberland Farms - Data Collection

Code written to collect latitude and longitude locations for Cumberland Farms using the [store location finder](https://www.cumberlandfarms.com/stores). The data is then cleaned with pandas and exported to a `.csv` file for use in the isochrone generation. There was 1 location error, `StoreId 6698` which had a latitude and longitude location that was in Greenland which was fixed for the cleaned dataset.

In [1]:
import json
import requests
import datetime as dt
import pandas as pd

In [2]:
# data collection
def request_data(url):
    r = requests.get(url)
    if r.status_code == 200:
        return r.json()
    else:
        print("[ERROR] 404 response")
        return
    
url = 'https://www.cumberlandfarms.com/storedata/getstoredatabylocation'

# run file
data = request_data(url)
df = pd.json_normalize(data)

# remove nulls
df = df[~df['StoreId'].isnull()]

# only those with ice coffee & save
df = df[df['IceCoffee'] == 'Y']
df = df.drop_duplicates(subset='StoreId')
df.to_csv('./data/files/cumberland_farms_raw.csv', index=False)

# wtf why is this mass gas station in greenland
df.loc[df['StoreId'] == '6698', 'Latitude'] = 42.246615
df.loc[df['StoreId'] == '6698', 'Longitude'] = -72.543669

# filter for wanted columns
df = df[['StoreId', 'Address', 'City', 'Latitude', 'Longitude']]
df.to_csv('./data/files/cumberland_farms_clean.csv', index=False)